In [1]:
import sys
sys.path.append("../")

In [2]:
from data_tools.containers import StocksData, StocksTarget
from data_tools.model_preprocessors import SequenceModelPreprocessor, StocksDataModelPreprocessor
from targets.target_function import ClosePriceTargetFunction
from models import LSTMRegressionModel
from features import IndicatorsFeaturesList

from sklearn.metrics import mean_absolute_percentage_error

c:\ProgramData\Anaconda3\envs\trading-algorithms\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
stocks_data = StocksData("../data/market_data/AAPL.csv")
stocks_data = IndicatorsFeaturesList().preprocess(stocks_data)

In [4]:
stocks_target = StocksTarget.from_target_function(stocks_data, ClosePriceTargetFunction(bars_count=3))

In [6]:
train_data, val_data, train_target, val_target = StocksDataModelPreprocessor.split(stocks_data, stocks_target)

preprocessor = SequenceModelPreprocessor(sequence_lenght=48)

X_train, y_train = preprocessor.preprocess(train_data, train_target)
X_val, y_val = preprocessor.preprocess(val_data, val_target)

In [7]:
model = LSTMRegressionModel(stocks_data.num_features())

model.fit(X_train, y_train, X_val, y_val, max_epochs=200)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                        | Params
-----------------------------------------------------------
0 | lstm       | LSTM                        | 2.2 M 
1 | linear     | Linear                      | 1.0 K 
2 | loss       | MSELoss                     | 0     
3 | train_mape | MeanAbsolutePercentageError | 0     
4 | val_mape   | MeanAbsolutePercentageError | 0     
-----------------------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.958     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\ProgramData\Anaconda3\envs\trading-algorithms\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


c:\ProgramData\Anaconda3\envs\trading-algorithms\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\ProgramData\Anaconda3\envs\trading-algorithms\lib\site-packages\pytorch_lightning\trainer\trainer.py:1595: PossibleUserWarning: The number of training batches (43) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 50:  41%|████      | 22/54 [00:13<00:20,  1.58it/s, loss=622, v_num=3, val_epoch_mape=0.203, train_epoch_mape=0.228]     